In [ ]:
import pandas as pd

data11 = pd.read_csv('../data/cian_sochi_all1-57_lat.csv')
data12 = pd.read_csv('../data/cian_sochi_all58-100_lat.csv')
data2 = pd.read_csv('../data/sochi_real_estate_full.csv')

data11['date'] = '2025-03-09'
data12['date'] = '2025-03-29'

data1 = pd.concat([data11, data12], ignore_index=True).drop_duplicates(subset=['url'])

data2 = data2.rename(columns={
    'geo_lat': 'latitude',
    'geo_lon': 'longitude',
    'level': 'floor',
    'levels': 'floors_count',
    'rooms': 'rooms_count',
    'area': 'total_meters'
})

common_cols = list(set(data1.columns) & set(data2.columns))
combined = pd.concat([data1[common_cols], data2[common_cols]], ignore_index=True)

combined['price'] = pd.to_numeric(combined['price'], errors='coerce')

combined['total_meters'] = pd.to_numeric(combined['total_meters'], errors='coerce')
combined['latitude'] = pd.to_numeric(combined['latitude'], errors='coerce')
combined['longitude'] = pd.to_numeric(combined['longitude'], errors='coerce')
combined = combined.dropna().drop_duplicates(subset=['latitude', 'longitude', 'date'])

combined['price_per_sqm'] = combined['price'] / combined['total_meters']

# we do not want to consider erroneous data
min_price = 1000.0

# avoiding overly luxurious stuff as outliers
max_price = 10000000.0

min_livespace = 10.0
combined = combined[(combined['price_per_sqm'] > min_price) & (combined['price_per_sqm'] < max_price)]
combined = combined[combined['total_meters'] > min_livespace]

In [ ]:
combined

In [ ]:
import overpy
import os

# if not os.path.exists('../data/critical_pois.csv'):
if True:
    bbox = [43.0, 39.0, 44.0, 40.5]
    bbox_str = ",".join(map(str, bbox))

    poi_queries = {
        "school": f'node["amenity"="school"]({bbox_str});',
        "bus_stop": f'node["highway"="bus_stop"]({bbox_str});',
        "park": f'node["leisure"="park"]({bbox_str});',
        "hospital": f'node["amenity"~"hospital|clinic"]({bbox_str});'
    }
    
    api = overpy.Overpass()
    # result = api.query(query)

    poi_coords = {}

    for poi_type, query_template in poi_queries.items():
        print(f"Querying OSM for {poi_type}s...")
        query = f"""
        [out:json][timeout:25];
        ({query_template});
        out center;
        """
        result = api.query(query)
        coords = [(float(node.lat), float(node.lon)) for node in result.nodes if hasattr(node, 'lat')]
        poi_coords[poi_type] = coords

    print("Querying OSM for coastline...")
    coastline_query = f"""
    [out:json][timeout:25];
    (
      way["natural"="coastline"]({bbox_str});
    );
    (._;>;);
    out body;
    """
    result = api.query(coastline_query)

    coastline_points = [
        (float(node.lat), float(node.lon))
        for way in result.ways
        for node in way.nodes
    ]

## Seeking the closest infrastructure elements (may take some time)

In [ ]:
serialized_file = '../data/poi_coords.pkl'

In [ ]:
poi_coords['center'] = [(43.57507244238309, 39.78258109486141)]

In [ ]:
poi_coords['sea']

In [ ]:
import pickle
with open(serialized_file, 'wb') as f:
    pickle.dump(poi_coords, f)

In [ ]:
from tqdm import tqdm
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

combined_coords = combined[['latitude', 'longitude']].values

for poi_type, poi_list in poi_coords.items():
    print(f"Calculating distances to nearest {poi_type}...")

    if not poi_list:
        # combined[f'dist_to_{poi_type}_km'] = np.nan
        print(f"Warning: no {poi_type}s were found")
        continue

    min_distances = []
    for lat, lon in tqdm(combined_coords, desc=f"Nearest {poi_type}"):
        if np.isnan(lat) or np.isnan(lon):
            min_distances.append(None)
            continue

        dists = [haversine(lat, lon, poi_lat, poi_lon) for poi_lat, poi_lon in poi_list]
        min_distances.append(min(dists))

    combined[f'dist_to_{poi_type}_km'] = min_distances

print("Calculating distance to sea...")
dist_to_sea = []
for lat, lon in tqdm(combined_coords, desc="Distance to sea"):
    if np.isnan(lat) or np.isnan(lon):
        dist_to_sea.append(None)
        continue
    dists = [haversine(lat, lon, sea_lat, sea_lon) for sea_lat, sea_lon in coastline_points]
    dist_to_sea.append(min(dists))

combined['dist_to_sea_km'] = dist_to_sea

In [ ]:
combined

In [ ]:
mortgage = pd.read_csv('../data/CBR_mortgage_data.csv')

# mortgage['Yearly rate (%)'] = mortgage['Yearly rate (%)'].str.replace(',', '.').astype(float)
# mortgage['New mortgages'] = mortgage['New mortgages'].str.replace(' ', '').astype(int)
# mortgage['New mortgage amount (millions)'] = mortgage['New mortgage amount (millions)'].str.replace(' ', '').astype(float)

# mortgage['date'] = pd.to_datetime(
#     mortgage['Year'].astype(str) + '-' + mortgage['Month'].astype(str) + '-01',
#     format='%Y-%m-%d'
# )

# mortgage = mortgage[['date', 'Yearly rate (%)', 'New mortgages', 'New mortgage amount (millions)']]
# mortage = mortgage.sort_values('date')
# mortage.to_csv('../data/CBR_mortgage_data.csv', index=False)
mortgage['inflation_multiplier'] = (1 + mortgage['Yearly rate (%)'] / 1200).cumprod()

In [ ]:
mortgage

In [ ]:
combined['date'] = pd.to_datetime(combined['date'])

merged = pd.merge_asof(
    combined.sort_values('date'),
    mortgage.sort_values('date'),
    on='date',
    direction='nearest'
)

merged['price_inflation_adjusted'] = merged['price'] / merged['inflation_multiplier']
merged['price_per_sqm_adjusted'] = merged['price_per_sqm'] / merged['inflation_multiplier']

merged = merged.dropna()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
geo_features = merged[['latitude', 'longitude', 'log_price_per_sqm']]
geo_scaled = scaler.fit_transform(geo_features)
kmeans = KMeans(n_clusters=5, random_state=42).fit(geo_scaled)
merged['cluster'] = kmeans.labels_

In [ ]:
merged

In [ ]:
merged.to_csv('../data/main_data.csv')